# Shock tube test

This notebook runs a shock-tube test with AsterX, and visualizes the data saved in OpenPMD format. 


### Test setup
This is a generic shock-tube test without magnetic fields, where the shock propagates along x-direction. The hydrodynamic portion of the initial conditions are the same as for the Balsara 1 shock tube problem. Here, the right and left states along x are initalized to different values.

- Left state is initialized as ($\rho, v^x, v^y, v^z, P$) = [1,0,0,1], and the right state as [0.125,0,0,0.1]. 

- For the ideal gas EOS, we consider $\gamma = 2$. 

- Initial data is set using the thorn ```HydroInitial```.

- Grid domain is $[-0.5,+0.5]$ along x, with 200 grid-cells.

- Flat boundary conditions are employed in all directions.

This gives the initial contact discontinuity at $x=0.0$.


## 1. Steps to perform the simulation

First, let's first move to the Cactus folder:

In [ ]:
cd ~/Cactus

Now, let's create the parameter file to be used for this simulation:

In [ ]:
%%bash
cat >./par/shocktube.par <<"#EOF"

ActiveThorns = "
    ADIOS2
    ADMBase
    AsterX
    CarpetX
    HydroBase
    HydroInitial
    IOUtil
    ODESolvers
    SystemTopology
    TimerReport
    TmunuBase
    openPMD
"

$nlevels = 1
$ncells = 200

Cactus::cctk_show_schedule = yes

Cactus::presync_mode = "mixed-error"

Cactus::terminate = "time"
Cactus::cctk_final_time = 0.40

ADMBase::set_adm_variables_during_evolution = "yes"
ADMBase::initial_data            = "Cartesian Minkowski"
ADMBase::initial_lapse           = "one"
ADMBase::initial_shift           = "zero"
ADMBase::initial_dtlapse         = "none"
ADMBase::initial_dtshift         = "none"

CarpetX::verbose = no

CarpetX::xmin = -0.5
CarpetX::ymin = -0.5 * 2 / $ncells
CarpetX::zmin = -0.5 * 2 / $ncells

CarpetX::xmax = +0.5
CarpetX::ymax = +0.5 * 2 / $ncells
CarpetX::zmax = +0.5 * 2 / $ncells

#flat bc
CarpetX::von_neumann_x =  yes
CarpetX::von_neumann_y =  yes
CarpetX::von_neumann_z =  yes
CarpetX::von_neumann_upper_x =  yes
CarpetX::von_neumann_upper_y =  yes
CarpetX::von_neumann_upper_z =  yes


CarpetX::ncells_x = $ncells
CarpetX::ncells_y = 2
CarpetX::ncells_z = 2

CarpetX::max_num_levels = $nlevels
CarpetX::regrid_every = 1000
CarpetX::blocking_factor_x = 1
CarpetX::blocking_factor_y = 1
CarpetX::blocking_factor_z = 1
CarpetX::regrid_error_threshold = 0.01

CarpetX::prolongation_type = "ddf"
CarpetX::ghost_size = 2
CarpetX::dtfac = 0.25

HydroBase::initial_hydro = "balsara1"
HydroInitial::gamma = 2.0
AsterX::gamma = 2.0
AsterX::reconstruction_method = "minmod"
AsterX::max_iter = 100

ODESolvers::method = "RK4"

IO::out_dir = $parfile
IO::out_every = 10 

CarpetX::out_openpmd_vars = "
    HydroBase::rho
    HydroBase::vel
    HydroBase::eps
    HydroBase::press
    CarpetX::regrid_error
"

TimerReport::out_every = 100 
TimerReport::out_filename = "TimerReport"
TimerReport::output_all_timers_together = yes
TimerReport::output_all_timers_readable = yes
TimerReport::n_top_timers = 50

#EOF

Then, submit the simulation using the following command:

In [ ]:
!./simfactory/bin/sim submit shocktube --parfile=./par/shocktube.par --config=sim-gpu --procs=1 --walltime=00:20:00

The above command creates and runs the simulation ```shocktube```, using the configuration ```sim-gpu```. The data is saved in the directory ```./simulations/shocktube```.



In [ ]:
# watch log output, following along as new output is produced
!./simfactory/bin/sim show-output --follow shocktube

## 2. Steps to visualize simulation data

The 2D data can be saved in both Silo format (which can be visualised, for instance, via VisIt) and in OpenPMD format. 

For further info on Silo, please visit: https://wci.llnl.gov/simulation/computer-codes/silo

For further info about OpenPMD, please visit:

- Official website:  https://www.openpmd.org
- GitHub repository: https://github.com/openPMD
- Documentation:     https://openpmd-api.readthedocs.io

Now, let's go back to the home directory:

In [ ]:
cd ~/

Import all the required modules:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from celluloid import Camera
import openpmd_api as io

Set the Matplotlib backend to `notebook`, not `inline`, since we'll want to animate some figures and the latter is not compatible with that

In [ ]:
%matplotlib notebook

Open a .bp file (ADIOS2 extension) as an OpenPMD **'series'**, which is a collection of **'iterations'**, each of which contains **'records'**, which are sets of either structured data --- **'meshes'** --- or unstructured data --- **'particles'**. 

AsterX only outputs mesh data. Each record has one or more **'components'**: for example, a record representing a scalar field only has one component, while a record representing a vector field has three.

In [ ]:
series = io.Series("./simulations/shocktube/output-0000/shocktube/shocktube.it%08T.bp", io.Access.read_only)

All iterations in our series have the same structure, i.e., they contain
the same records, since they all represent the same output, just at
different times. 

Here we define an empty Python nested dictionary whose
structure, once full, will be:

Iteration 0:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

Iteration 1:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

[...]

In [ ]:
iter_rec_comp_dict = {}

Print info, register data chunks and fill the above dictionary:

In [ ]:
for index in series.iterations:
    iteration = str(index)

    print("\nIteration " + iteration + ":")
    print("==============")

    # Allocate an empty dictionary associated to this iteration
    iter_rec_comp_dict[iteration] = {}

    i = series.iterations[index]

    for key in i.meshes:
        print("Components of record \"" + key + "\":")

        # Allocate an empty dictionary associated to this record. Notice that
        # 'record' is an OpenPMD mesh object, so it's better to use 'key'
        # instead of 'record' as a key in the dictionary ('record' could also be
        # used, but it makes accessing the key clumsy).
        record = i.meshes[key]
        iter_rec_comp_dict[iteration][key] = {}

        # Load each component of each record as a 'data chunk', i.e., an
        # allocated, but STILL INVALID, NumPy array. Later we will flush all
        # chunks (i.e., basically, fill the NumPy arrays) at once: this leads
        # to better I/O performance compared to flushing a large number of
        # small chunks. That's why we bothered creating the nested dictionary:
        # in this way, we can access the valid NumPy arrays for plotting
        # without having to flush each single chunk.
        # *IMPORTANT*: DO NOT access data chunks until flushing has happened!
        for component in record:
            print("    > " + component)  # 'component' is a string
            iter_rec_comp_dict[iteration][key][component] = record[component].load_chunk()  # *INVALID* 3D NumPy array

        print("")

Flush all registered data chunks, which are now **VALID** 3D NumPy arrays:

In [ ]:
series.flush()

Visualize a 2D movie of the mass density on the xz plane:

In [ ]:
# Select the desired record and component to plot
record    = "hydrobase_rho_rl00"  # "carpetx_regrid_error_rl00", "hydrobase_eps_rl00", "hydrobase_press_rl00", "hydrobase_rho_rl00", "hydrobase_vel_rl00"
component = "hydrobase_rho"  # "carpetx_regrid_error", "hydrobase_eps", "hydrobase_press", "hydrobase_rho", "hydrobase_velx", "hydrobase_vely", "hydrobase_velz"

# Set up the axes for the plot and the colorbar
fig    = plt.figure(figsize = [5.0, 2.75])
axplot = fig.add_axes([0.12, 0.14, 0.75, 0.75])
axclb  = fig.add_axes([0.88, 0.14, 0.02, 0.75])

# Set title and labels
axplot.set_title("Rest-mass density", fontsize = 10., fontweight = "bold", color = "midnightblue")
axplot.set_xlabel("x", fontsize = 7.)
axplot.set_ylabel("z", fontsize = 7.)
axplot.tick_params(labelsize=7)
axplot.xaxis.set_major_locator(plt.MaxNLocator(5))
axplot.yaxis.set_major_locator(plt.MaxNLocator(5))


# Initialize the camera
camera = Camera(fig)

# Print frames
for iteration in iter_rec_comp_dict:
    # Retrieve the 3D array containing the data
    array3D = iter_rec_comp_dict[iteration][record][component]
    
    # Plot on the (x, z) plane at the half-way value of y
    # Notice that the 3D array is stored in (z, y, x) order
    y_index = int(array3D.shape[1]/2)
    x0     = np.linspace(-0.5, 0.5, array3D.shape[2])
    z0     = np.linspace(-0.04, 0.04, array3D.shape[0])
    image   = axplot.pcolormesh(x0, z0, array3D[:, y_index, :],
                                cmap = "magma", vmin = 0.0, vmax = 1.0)
    axplot.set_ylim(ymin=-0.005, ymax=0.005)
    # Set up the colorbar
    axclb.tick_params(labelsize=7.0)
    fig.colorbar(image, cax = axclb, extend = "neither")
    
    # Print the current iteration
    axplot.text(0.18, 0.42, "Iteration " + iteration,
             fontsize = 8., fontweight = "bold", color = "white")

    # Take a snapshot of the figure at this iteration (needed later for the animation)
    camera.snap()

In [ ]:
from IPython.display import HTML
animation = camera.animate()
HTML(animation.to_html5_video())